In [ ]:
import os
import pandas as pd
from google.cloud import bigquery
import db_dtypes

#scenic-outcome-453219-h5-4c318c9a7175
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="scenic-outcome-453219-h5-4c318c9a7175.json"
client = bigquery.Client() 

pd.set_option('display.max_columns', None)  # wyłącz limit na kolumny
pd.set_option('display.max_rows', None)  # wyłącz limit na wiersze
pd.set_option('display.width', None)  # wyłącz limit na szerokość
pd.set_option('display.max_colwidth', None)  # wyłącz limit na szerokość kolumn
# client = bigquery.Client()

In [ ]:
 query = ('select * from bigquery-public-data.covid19_open_data.covid19_open_data limit 20')
 query_job = client.query(query)    
 query_result = query_job.result()  
 df = query_result.to_dataframe()

In [ ]:
df

In [ ]:
#Lista wszystkich kolumn
query = """
    SELECT column_name
    FROM `bigquery-public-data.covid19_open_data.INFORMATION_SCHEMA.COLUMNS`
    WHERE table_name = 'covid19_open_data'
"""
query_job = client.query(query)
columns_df = query_job.result().to_dataframe()

columns_df